NHK’s Program List API allows us to get title and subtitle for all the shows scheduled for the next 7 days for TV, Radio, and Internet Radio all across Japan.

In [1]:
# In this notebook, I am getting show list for 7/11/22 - 7/17/22, then concat with the previous two weeks' data

In [1]:
import pandas as pd
import json
import requests
# import datetime

# APIキーをセット
apikey = '9h1ARm6G4NktGZgaR2BDGclcsN181dDZ'

# 130:東京
area = '130'
# g1:NHK総合１,e1:Eテレ,s1:BS1,s3:BSP,r1:ラジオ第1, r1:ラジオ第2
service = ['g1','e1','e4','s1','s3','r1','r2','r3']
# date/time
date = ['2022-07-10','2022-07-11','2022-07-12','2022-07-13','2022-07-14','2022-07-15','2022-07-16','2022-07-17'] 
# date = datetime.date.today()+timedelta(days=8)

all_results = pd.DataFrame(index=None, columns=[])

for j in range(len(date)):
    for i in range(len(service)):
        url = 'https://api.nhk.or.jp/v2/pg/list/{0}/{1}/{2}.json?key={3}'.format(area,service[i],date[j],apikey)
        request_get = requests.get(url)
        if request_get.status_code != 200:
            print('NHK番組表APIのデータが取得出来ません。')
            break
        result = pd.json_normalize(request_get.json(), ['list',[service[i]]])
        all_results = pd.concat([all_results, result])
    

all_results = all_results[~all_results['title'].str.contains('放送休止')]

NHK番組表APIのデータが取得出来ません。
NHK番組表APIのデータが取得出来ません。


In [2]:
all_results.shape

(3033, 22)

In [3]:
all_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3033 entries, 0 to 41
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     3033 non-null   object
 1   event_id               3033 non-null   object
 2   start_time             3033 non-null   object
 3   end_time               3033 non-null   object
 4   title                  3033 non-null   object
 5   subtitle               3033 non-null   object
 6   content                3033 non-null   object
 7   act                    3033 non-null   object
 8   genres                 3033 non-null   object
 9   area.id                3033 non-null   object
 10  area.name              3033 non-null   object
 11  service.id             3033 non-null   object
 12  service.name           3033 non-null   object
 13  service.logo_s.url     3033 non-null   object
 14  service.logo_s.width   3033 non-null   object
 15  service.logo_s.height  

In [4]:
all_results.to_csv(r'nhk_7-10-22.csv', index = False, header=True)

In [5]:
all_results.head(1)

,id,event_id,start_time,end_time,title,subtitle,content,act,genres,area.id,...,service.name,service.logo_s.url,service.logo_s.width,service.logo_s.height,service.logo_m.url,service.logo_m.width,service.logo_m.height,service.logo_l.url,service.logo_l.width,service.logo_l.height
0,2022071007199,07199,2022-07-10T04:13:00+09:00,2022-07-10T04:15:00+09:00,インターミッション,,,,[0815],130,...,ＮＨＫ総合１,//www.nhk.or.jp/common/img/media/gtv-100x50.png,100,50,//www.nhk.or.jp/common/img/media/gtv-200x100.png,200,100,//www.nhk.or.jp/common/img/media/gtv-200x200.png,200,200


In [7]:
# importing last two weeks' data
df_0626_0703 = pd.read_csv("nhk_0626_0703.csv")

In [8]:
# combining with this week's data
df_0626_0703_0710 = pd.concat([df_0626_0703, all_results], axis=0)

In [9]:
df_0626_0703_0710.shape

(9695, 22)

In [10]:
df_0626_0703_0710.tail()

,id,event_id,start_time,end_time,title,subtitle,content,act,genres,area.id,...,service.name,service.logo_s.url,service.logo_s.width,service.logo_s.height,service.logo_m.url,service.logo_m.width,service.logo_m.height,service.logo_l.url,service.logo_l.width,service.logo_l.height
37,2022071611438,11438,2022-07-17T02:00:00+09:00,2022-07-17T03:00:00+09:00,ヨーロッパ トラムの旅「チェコ プラハ」,欧州各地の都市を走る路面電車・トラム。車窓から見る美しいヨーロッパの街並み。街の名所旧跡も巡...,欧州各地の都市を走る路面電車・トラム。どこか懐かしくレトロなたたずまいは、旅人をひきつけてや...,,"[0801, 1000, 0807]",130,...,ＮＨＫＢＳ１,//www.nhk.or.jp/common/img/media/bs1-100x50.png,100,50,//www.nhk.or.jp/common/img/media/bs1-200x100.png,200,100,//www.nhk.or.jp/common/img/media/bs1-200x200.png,200,200
38,2022071611439,11439,2022-07-17T03:00:00+09:00,2022-07-17T04:00:00+09:00,ヨーロッパ トラムの旅「セルビア ベオグラード」,欧州各地の都市を走る路面電車・トラム。車窓から見る美しいヨーロッパの街並み。街の名所旧跡も巡...,欧州各地の都市を走る路面電車・トラム。どこか懐かしくレトロなたたずまいは、旅人をひきつけて止...,,"[0801, 1000, 0807]",130,...,ＮＨＫＢＳ１,//www.nhk.or.jp/common/img/media/bs1-100x50.png,100,50,//www.nhk.or.jp/common/img/media/bs1-200x100.png,200,100,//www.nhk.or.jp/common/img/media/bs1-200x200.png,200,200
39,2022071611440,11440,2022-07-17T04:00:00+09:00,2022-07-17T04:30:00+09:00,映像詩「祈りの桜 奈良県吉野山」,奈良在住の映像作家・保山耕一さんが撮影した吉野山の桜。風に舞う花びらと光が織り成す、祈りの光景。,,,"[0801, 0802]",130,...,ＮＨＫＢＳ１,//www.nhk.or.jp/common/img/media/bs1-100x50.png,100,50,//www.nhk.or.jp/common/img/media/bs1-200x100.png,200,100,//www.nhk.or.jp/common/img/media/bs1-200x200.png,200,200
40,2022071611441,11441,2022-07-17T04:30:00+09:00,2022-07-17T04:59:00+09:00,Ｊｏｕｒｎｅｙｓ ｉｎ Ｊａｐａｎ「鈴鹿 サーキットとともに」,日本屈指のレーシングコースがあり、モータースポーツの街として知られる三重県鈴鹿市。仏ル・マン...,▽鈴鹿サーキットはレーシングコース以外に遊園地やホテルもある複合施設。本物のコースを電動カー...,【リポーター】ジェニファー・ジュリアン，【語り】織江珠生，ビル・サリバン,"[0801, 0804, 0807]",130,...,ＮＨＫＢＳ１,//www.nhk.or.jp/common/img/media/bs1-100x50.png,100,50,//www.nhk.or.jp/common/img/media/bs1-200x100.png,200,100,//www.nhk.or.jp/common/img/media/bs1-200x200.png,200,200
41,2022071611444,11444,2022-07-17T04:59:00+09:00,2022-07-17T05:00:00+09:00,河合克敏×視覚障害者柔道×ｍｉｗａ アニ×パラ第６弾,パラスポーツをアニメで描く「アニ×パラ～視覚障害者柔道編」のテーマ曲版。ｍｉｗａの歌に乗せて...,,【原作】河合克敏，【監督】前田薫平，【出演】ｍｉｗａ，広瀬順子，【語り】上白石萌音,"[0700, 0106, 0105]",130,...,ＮＨＫＢＳ１,//www.nhk.or.jp/common/img/media/bs1-100x50.png,100,50,//www.nhk.or.jp/common/img/media/bs1-200x100.png,200,100,//www.nhk.or.jp/common/img/media/bs1-200x200.png,200,200


In [12]:
df_0626_0703_0710.to_csv(r'nhk_0626_0703_0710.csv', index = False, header=True)